In [1]:

import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [2]:
csv = 'clean_tweet.csv'
my_df = pd.read_csv(csv,index_col=0)
my_df.head()

C:\Users\admin\Anaconda2\lib\site-packages\numpy\lib\arraysetops.py:522: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


,text,target
0,awww that bummer you shoulda got david carr of...,0
1,is upset that he can not update his facebook b...,0
2,dived many times for the ball managed to save ...,0
3,my whole body feels itchy and like its on fire,0
4,no it not behaving at all mad why am here beca...,0


In [3]:
my_df.dropna(inplace=True)
my_df.reset_index(drop=True,inplace=True)
my_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1596019 entries, 0 to 1596018
Data columns (total 2 columns):
text      1596019 non-null object
target    1596019 non-null int64
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [4]:

x = my_df.text
y = my_df.target

In [5]:
 from sklearn.model_selection import train_test_split
SEED = 2000
x_train, x_validation_and_test, y_train, y_validation_and_test = train_test_split(x, y, test_size=.02, random_state=SEED)
x_validation, x_test, y_validation, y_test = train_test_split(x_validation_and_test, y_validation_and_test, test_size=.5, random_state=SEED)

In [6]:
from sklearn.linear_model import LogisticRegression

In [7]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from gensim.models.doc2vec import LabeledSentence
import multiprocessing
from sklearn import utils

In [8]:
def labelize_tweets_ug(tweets,label):
    result = []
    prefix = label
    for i, t in zip(tweets.index, tweets):
        result.append(LabeledSentence(t.split(), [prefix + '_%s' % i]))
    return result

In [9]:
all_x = pd.concat([x_train,x_validation,x_test])
all_x_w2v = labelize_tweets_ug(all_x, 'all')

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `LabeledSentence` (Class will be removed in 4.0.0, use TaggedDocument instead).
  """


In [10]:

len(all_x_w2v)

1596019

In [11]:
model_ug_dbow = Doc2Vec(dm=0, vector_size=100, negative=5, min_count=2, alpha=0.065, min_alpha=0.065)
model_ug_dbow.build_vocab([x for x in tqdm(all_x_w2v)])

100%|██████████| 1596019/1596019 [00:00<00:00, 4721948.88it/s]


In [12]:

%%time
for epoch in range(30):
    model_ug_dbow.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dbow.alpha -= 0.002
    model_ug_dbow.min_alpha = model_ug_dbow.alpha

100%|██████████| 1596019/1596019 [00:00<00:00, 4666722.12it/s]


Wall time: 22min 39s


In [13]:

def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

In [14]:
train_vecs_dbow = get_vectors(model_ug_dbow, x_train, 100)
validation_vecs_dbow = get_vectors(model_ug_dbow, x_validation, 100)

In [15]:
clf = LogisticRegression()
clf.fit(train_vecs_dbow, y_train)

C:\Users\admin\Anaconda2\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [16]:
clf.score(validation_vecs_dbow, y_validation)

0.7337719298245614

In [17]:

model_ug_dbow.save('d2v_model_ug_dbow.doc2vec')
model_ug_dbow = Doc2Vec.load('d2v_model_ug_dbow.doc2vec')

In [18]:
model_ug_dbow.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [19]:
cores = multiprocessing.cpu_count()
model_ug_dmc = Doc2Vec(dm=1, dm_concat=1, size=100, window=2, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmc.build_vocab([x for x in tqdm(all_x_w2v)])

C:\Users\admin\Anaconda2\lib\site-packages\gensim\models\doc2vec.py:580: UserWarning: The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.
  warnings.warn("The parameter `size` is deprecated, will be removed in 4.0.0, use `vector_size` instead.")
100%|██████████| 1596019/1596019 [00:00<00:00, 4586263.98it/s]


In [ ]:
%%time
for epoch in range(10):
    model_ug_dmc.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmc.alpha -= 0.002
    model_ug_dmc.min_alpha = model_ug_dmc.alpha

In [21]:

def get_vectors(model, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = model.docvecs[prefix]
        n += 1
    return vecs

In [22]:
train_vecs_dmc = get_vectors(model_ug_dmc, x_train, 100)
validation_vecs_dmc = get_vectors(model_ug_dmc, x_validation, 100)

In [23]:
model_ug_dmc.save('d2v_model_ug_dmc.doc2vec')
model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')

In [24]:

model_ug_dmc.most_similar('good')

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


[('goood', 0.6508709192276001),
 ('gd', 0.6128671169281006),
 ('goooood', 0.6113631725311279),
 ('gooooood', 0.5972696542739868),
 ('betr', 0.5952520966529846),
 ('goooooooooooooood', 0.5893263220787048),
 ('depresing', 0.5882762670516968),
 ('goooooood', 0.5868667364120483),
 ('goooooooooood', 0.5827385187149048),
 ('rough', 0.5819293260574341)]

In [25]:
model_ug_dmc.most_similar('facebook')

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('bebo', 0.7541865706443787),
 ('myspace', 0.7146177291870117),
 ('weebly', 0.6938952207565308),
 ('mypace', 0.688728928565979),
 ('youtube', 0.6831573247909546),
 ('mileyloverx', 0.6809428334236145),
 ('thirsday', 0.6650829315185547),
 ('skype', 0.6600528955459595),
 ('ustream', 0.6598027348518372),
 ('xtube', 0.6510134339332581)]

In [26]:
model_ug_dmc.most_similar(positive=['bigger', 'small'], negative=['big'])

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('sharper', 0.5563883185386658),
 ('poorer', 0.5541378259658813),
 ('saner', 0.5499848127365112),
 ('shorter', 0.5388410687446594),
 ('slower', 0.5307089686393738),
 ('dorkier', 0.5285966992378235),
 ('tastier', 0.5283513069152832),
 ('smaller', 0.5215487480163574),
 ('deadlier', 0.5105226039886475),
 ('drunker', 0.509262204170227)]

In [27]:
clf = LogisticRegression()
clf.fit(train_vecs_dmc, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [28]:
clf.score(validation_vecs_dmc, y_validation)

0.64968671679198

In [29]:
model_ug_dmc.save('d2v_model_ug_dmc.doc2vec')
model_ug_dmc = Doc2Vec.load('d2v_model_ug_dmc.doc2vec')
model_ug_dmc.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [30]:
cores = multiprocessing.cpu_count()
model_ug_dmm = Doc2Vec(dm=1, dm_mean=1, size=100, window=4, negative=5, min_count=2, workers=cores, alpha=0.065, min_alpha=0.065)
model_ug_dmm.build_vocab([x for x in tqdm(all_x_w2v)])

100%|██████████| 1596019/1596019 [00:00<00:00, 4313566.20it/s]


In [31]:
%%time
for epoch in range(20):
    model_ug_dmm.train(utils.shuffle([x for x in tqdm(all_x_w2v)]), total_examples=len(all_x_w2v), epochs=1)
    model_ug_dmm.alpha -= 0.002
    model_ug_dmm.min_alpha = model_ug_dmm.alpha

100%|██████████| 1596019/1596019 [00:00<00:00, 4233471.10it/s]


Wall time: 21min 2s


In [32]:
model_ug_dmm.most_similar('good')

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('goood', 0.673827052116394),
 ('nice', 0.6542283296585083),
 ('great', 0.6522321105003357),
 ('tough', 0.6460471749305725),
 ('decent', 0.6294572949409485),
 ('cool', 0.6249179244041443),
 ('gud', 0.6144165992736816),
 ('goooood', 0.6112074851989746),
 ('brilliant', 0.5999947786331177),
 ('fab', 0.5942299365997314)]

In [33]:
model_ug_dmc.most_similar('happy')

C:\Users\admin\Anaconda2\lib\site-packages\ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  """Entry point for launching an IPython kernel.


[('hapy', 0.7078632116317749),
 ('happpppy', 0.656713604927063),
 ('janell', 0.6355280876159668),
 ('happpy', 0.6305738091468811),
 ('happyyy', 0.6067249774932861),
 ('happpppppy', 0.59429931640625),
 ('teuni', 0.5942220687866211),
 ('thrilled', 0.5890579223632812),
 ('pleased', 0.5859392881393433),
 ('pias', 0.5855867862701416)]

In [34]:
train_vecs_dmm = get_vectors(model_ug_dmm, x_train, 100)
validation_vecs_dmm = get_vectors(model_ug_dmm, x_validation, 100)

In [35]:
clf = LogisticRegression()
clf.fit(train_vecs_dmm, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [36]:
clf.score(validation_vecs_dmm, y_validation)

0.7271303258145363

In [37]:
model_ug_dmm.save('d2v_model_ug_dmm.doc2vec')
model_ug_dmm = Doc2Vec.load('d2v_model_ug_dmm.doc2vec')
model_ug_dmm.delete_temporary_training_data(keep_doctags_vectors=True, keep_inference=True)

In [38]:
def get_concat_vectors(model1,model2, corpus, size):
    vecs = np.zeros((len(corpus), size))
    n = 0
    for i in corpus.index:
        prefix = 'all_' + str(i)
        vecs[n] = np.append(model1.docvecs[prefix],model2.docvecs[prefix])
        n += 1
    return vecs

In [39]:
train_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_train, 200)
validation_vecs_dbow_dmc = get_concat_vectors(model_ug_dbow,model_ug_dmc, x_validation, 200)

In [40]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmc, y_train)

Wall time: 44.2 s


In [41]:
clf.score(validation_vecs_dbow_dmc, y_validation)

0.7399122807017544

In [42]:
train_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_train, 200)
validation_vecs_dbow_dmm = get_concat_vectors(model_ug_dbow,model_ug_dmm, x_validation, 200)

In [43]:
%%time
clf = LogisticRegression()
clf.fit(train_vecs_dbow_dmm, y_train)

Wall time: 58.8 s


In [44]:
clf.score(validation_vecs_dbow_dmm, y_validation)

0.7505012531328321